# Lab 02: Logistic Regression

### 1.1 Data

Data dùng trong bài thực hành này gồm chiều dài dọc gân lá ($x_1$) và chiều ngang gân lá ($x_2$) và loại lá ($y$, có giá trị 0 hoặc 1).


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

train = pd.read_csv("train.csv")
train.head()

x1         x2  label
0  56.130749  61.006677      0
1  49.871157  43.687531      0
2  53.868769  62.477624      0
3  53.558846  52.686585      0
4  55.382830  52.925488      0

In [7]:
x1_train = train['x1'].values
x2_train = train['x2'].values
y_train = train['label'].values

plt.scatter(x1_train[y_train == 0], x2_train[y_train==0])
plt.scatter(x1_train[y_train == 1], x2_train[y_train==1])


plt.figure()
valid = pd.read_csv("valid.csv")
x1_valid = valid['x1'].values
x2_valid = valid['x2'].values
y_valid = valid['label'].values
plt.scatter(x1_valid[y_valid == 0], x2_valid[y_valid==0])
plt.scatter(x1_valid[y_valid == 1], x2_valid[y_valid==1])


### 1.2 Binary Logistic Linear Regression
Ta muốn dựa vào 2 chiều của lá, phân biệt giữa loại lá 1 và loại lá 2. Cụ thể, với $x = (x_1,x_2, 1)$ là input, ta muốn đoán một phân phối
$$ P_\theta(c|x),c = 0, 1 $$
với $\theta = (\theta_1, \theta_2, \theta_3)$ là tham số.

Trong binary losgistic linear regression, ta giả thuyết
$$ P_\theta(c=0|x) = \frac{1}{1+ e^{-\theta_1 x_1 - \theta_2 x_2 - \theta_3}} = \frac{1}{1+ e^{-\theta \cdot x}} \text{ và } P_\theta(c=1|x) = 1 - P_\theta(c=0|x)$$

Giờ cần tìm $\theta = (\theta_1, \theta_2, \theta_3)$ sao cho 
$$ L = \frac{1}{N} \sum_{x, y} - (1-y) \log P_\theta(c=0|x) - y \log P_\theta(c=1|x) = \frac{1}{N} \sum_{x, y} - (1-y) \log \frac{1}{1+ e^{-\theta \cdot x}} - y \log \frac{e^{-\theta \cdot x}}{1+ e^{-\theta \cdot x}} $$
đạt giá trị nhỏ nhất. 

Nếu dùng Gradient Descent:
$$ \frac{\partial L}{\partial \theta_1} = \frac{1}{N} \sum_{x, y} e^{-\theta \cdot x}  \left( \frac{1}{1+e^{-\theta \cdot x}} - \frac{y}{e^{-\theta \cdot x}}\right) (-x_1)$$
$$ \frac{\partial L}{\partial \theta_2} = \frac{1}{N} \sum_{x, y} e^{-\theta \cdot x}  \left( \frac{1}{1+e^{-\theta \cdot x}} - \frac{y}{e^{-\theta \cdot x}}\right) (-x_2)$$
$$ \frac{\partial L}{\partial \theta_3} = \frac{1}{N} \sum_{x, y} e^{-\theta \cdot x}  \left( \frac{1}{1+e^{-\theta \cdot x}} - \frac{y}{e^{-\theta \cdot x}}\right) (-1) $$

In [17]:
theta1, theta2, theta3 = 0.5, 0.5, 0.5
lrn_rate = 0.001
n_iters = 10000

best_theta = None      #biến để lưu tham số mà có accuracy tốt nhất trên tập valid
best_val_acc = -1.0

for ii in range(n_iters):
    exppp = np.exp(-theta1*x1_train -theta2*x2_train-theta3)
    common = exppp*(1/(1+exppp) - y_train/exppp)
    dL_dtheta1 = np.mean(common*(-x1_train))
    dL_dtheta2 = np.mean(common*(-x2_train))
    dL_dtheta3 = np.mean(common*(-1))
    theta1 = theta1 - lrn_rate * dL_dtheta1
    theta2 = theta2 - lrn_rate * dL_dtheta2
    theta3 = theta3 - lrn_rate * dL_dtheta3
    
    if ii % 500 == 0:
        ## predict
        pred_proba_train = 1/(1+np.exp(-theta1*x1_train-theta2*x2_train-theta3))            ## xác suất label 0
        train_loss = np.mean(-(1-y_train)*np.log(pred_proba_train) - y_train*np.log(1-pred_proba_train))
        train_acc = np.mean(y_train == (pred_proba_train < 0.5).astype(int))                ## convert xác suất đoán đc ra nhãn rồi tính accuracy
        
        
        pred_proba_valid = 1/(1+np.exp(-theta1*x1_valid-theta2*x2_valid-theta3))
        valid_loss = np.mean(-(1-y_valid)*np.log(pred_proba_valid) - y_valid*np.log(1-pred_proba_valid))
        valid_acc = np.mean(y_valid == (pred_proba_valid < 0.5).astype(int))
        
        if valid_acc > best_val_acc:
            ## lưu tham số tốt nhất
            best_val_acc = valid_acc
            best_theta = theta1, theta2, theta3
        
        print("Iter {} - loss {:.4f} - acc {:.4f} - val_loss {:.4f} - val_acc {:.4f}".format(ii, train_loss, train_acc, valid_loss, valid_acc))

theta1, theta2, theta3 = best_theta
train_acc = np.mean(y_train == (1/(1+np.exp(-theta1*x1_train-theta2*x2_train-theta3)) < 0.5).astype(int))
valid_acc = np.mean(y_valid == (1/(1+np.exp(-theta1*x1_valid-theta2*x2_valid-theta3)) < 0.5).astype(int))
print("Accuracy on train: ", train_acc)
print("Accuracy on valid: ", valid_acc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in multiply


Iter 0 - loss nan - acc 0.5000 - val_loss nan - val_acc 0.5000
Iter 500 - loss 0.3738 - acc 0.8567 - val_loss 0.4882 - val_acc 0.7750
Iter 1000 - loss 0.3453 - acc 0.8550 - val_loss 0.4848 - val_acc 0.7750
Iter 1500 - loss 0.3364 - acc 0.8550 - val_loss 0.4898 - val_acc 0.7800
Iter 2000 - loss 0.3327 - acc 0.8567 - val_loss 0.4950 - val_acc 0.7800
Iter 2500 - loss 0.3310 - acc 0.8567 - val_loss 0.4994 - val_acc 0.7800
Iter 3000 - loss 0.3301 - acc 0.8567 - val_loss 0.5030 - val_acc 0.7800
Iter 3500 - loss 0.3296 - acc 0.8567 - val_loss 0.5057 - val_acc 0.7800
Iter 4000 - loss 0.3293 - acc 0.8567 - val_loss 0.5078 - val_acc 0.7800
Iter 4500 - loss 0.3292 - acc 0.8567 - val_loss 0.5095 - val_acc 0.7800
Iter 5000 - loss 0.3291 - acc 0.8567 - val_loss 0.5108 - val_acc 0.7800
Iter 5500 - loss 0.3291 - acc 0.8567 - val_loss 0.5117 - val_acc 0.7800
Iter 6000 - loss 0.3290 - acc 0.8567 - val_loss 0.5125 - val_acc 0.7800
Iter 6500 - loss 0.3290 - acc 0.8567 - val_loss 0.5131 - val_acc 0.7800
It

In [13]:
theta1, theta2, theta3

(-0.0011279556179452856, 0.00119213803349409, 0.0)

In [16]:
theta1, theta2, theta3

(-0.45554812117023324, 0.47906067555940474, 0.9796579021921025)

In [18]:
theta1, theta2, theta3

(-0.3769571176096302, 0.4030675155314635, 0.49017351825774713)

In [ ]:
# Tìm được delta(delta_1, delta_2, delta_3)
# Thử với: - delta(-delta_1, -delta_2, -delta_3)

### 1.3 Sklearn

In [9]:
from sklearn.linear_model import LogisticRegression

X_train = np.concatenate([x1_train.reshape(-1,1), x2_train.reshape(-1,1)], axis=-1)
X_valid = np.concatenate([x1_valid.reshape(-1,1), x2_valid.reshape(-1,1)], axis=-1)

lr = LogisticRegression().fit(X_train, y_train)
print("Sklearn accuracy on train: ", np.mean(y_train == lr.predict(X_train)))
print("Sklearn accuracy on valid: ", np.mean(y_valid == lr.predict(X_valid)))

Sklearn accuracy on train:  0.8566666666666667
Sklearn accuracy on valid:  0.775


In [ ]:
help(lr)

In [22]:
lr.coef_, lr.intercept_

(array([[ 0.48472337, -0.51235281]]), array([-0.91670467]))